In [1]:
import wandb
import json
import yaml
import os, sys

In [2]:
wb_config_name = 'wb_config_1.yaml'
wb_project = 'videomae_finetuning'

sweep_id_dict_name = 'sweep_id_dict.json'

In [3]:
with open(sweep_id_dict_name) as f:
    sweep_dict = json.load(f)
    if wb_config_name in sweep_dict:
        sweep_id = sweep_dict[wb_config_name]
    else:
        # Initialize sweep by passing in config
        config = yaml.load(open(wb_config_name), Loader=yaml.Loader)
        sweep_id = wandb.sweep(sweep=config,
                               project=wb_project,
                               entity='flatiron-scipt')
        sweep_dict[wb_config_name] = sweep_id
        with open(sweep_id_dict_name, 'w') as f:
            json.dump(sweep_dict, f, indent=4)
        print(f"New sweep_id created for config {wb_config_name}: {sweep_id}. Added to the sweep_ids dictionary!")
        sys.exit(0)

In [4]:
with open('finetune_template.sh', 'r') as f:
    finetune_script_template = f.read()

In [28]:
run_id_start = 0
nb_runs = 50
for run_id in range(run_id_start, run_id_start + nb_runs):
    finetune_script = finetune_script_template.replace('{{run_id}}', str(run_id))
    finetune_script = finetune_script.replace('{{sweep_id}}', sweep_id)
    with open(f'jobs/finetune_{run_id}.sh', 'w') as f:
        f.write(finetune_script)
    os.system(f'sbatch jobs/finetune_{run_id}.sh')

Submitted batch job 2620649
